In [ ]:

!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 16.0 MB/s 
     |████████████████████████████████| 182 kB 62.0 MB/s 
     |████████████████████████████████| 7.6 MB 45.4 MB/s 


In [7]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import transformers
from transformers import AutoModel, BertTokenizerFast

# specify GPU
device = torch.device("cuda")
     

In [8]:

# import BERT-base pretrained model
bert = AutoModel.from_pretrained('bert-base-uncased')

# Load the BERT tokenizer
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Extract trainig Data and labels

In [15]:
df = pd.read_csv("train.csv")
df.head()

,text,category,stance
0,بيل غيتس يتلقى لقاح #كوفيد19 من غير تصوير الاب...,celebrity,1
1,وزير الصحة لحد اليوم وتحديدا هلأ بمؤتمروا الصح...,info_news,1
2,قولكن رح يكونو اد المسؤولية ب لبنان لما يوصل ...,info_news,1
3,#تركيا.. وزير الصحة فخر الدين قوجة يتلقى أول ج...,celebrity,1
4,وئام وهاب يشتم الدول الخليجية في كل طلة اعلامي...,personal,0


In [16]:
df.category.value_counts()

info_news       3616
personal        1025
celebrity        975
plan             606
unrelated        323
others           167
requests         112
rumors            79
advice            67
restrictions      18
Name: category, dtype: int64

In [17]:
# store possible unique category
possible_labels = df.category.unique()

In [18]:
print(possible_labels)

['celebrity' 'info_news' 'personal' 'unrelated' 'plan' 'requests' 'others'
 'rumors' 'advice' 'restrictions']


In [21]:
# give each category number from 0 -> 9
label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index


In [22]:
label_dict

{'celebrity': 0,
 'info_news': 1,
 'personal': 2,
 'unrelated': 3,
 'plan': 4,
 'requests': 5,
 'others': 6,
 'rumors': 7,
 'advice': 8,
 'restrictions': 9}

In [23]:
# replace each category in our data frame wit
df.category = df['category'].map(label_dict)

In [46]:
df.head(10)

,text,category,stance,data_type
0,بيل غيتس يتلقى لقاح #كوفيد19 من غير تصوير الاب...,0,1,train
1,وزير الصحة لحد اليوم وتحديدا هلأ بمؤتمروا الصح...,1,1,train
2,قولكن رح يكونو اد المسؤولية ب لبنان لما يوصل ...,1,1,train
3,#تركيا.. وزير الصحة فخر الدين قوجة يتلقى أول ج...,0,1,val
4,وئام وهاب يشتم الدول الخليجية في كل طلة اعلامي...,2,0,train
5,"لقاح #كورونا في أميركا.. قلق متزايد من ""التوزي...",1,0,train
6,لبنان اشترى مليونان لقاح امريكي اذا شلنا يلي ع...,1,1,train
7,من عوارض لقاح كورونا<LF>هو تهكير حسابك عتويتر<...,2,0,train
8,هناك 1780 مليونيراً في لبنان. ماذا لو فُرضت ال...,3,0,train
9,دعبول حضرتك منو انت وتطلب من قائد دولة إسلامية...,1,1,train


In [39]:
# X_train, X_val, y_train, y_val = train_test_split(df.index.values, 
#                                                   df.category.values, 
#                                                   test_size=0.15, 
#                                                   random_state=42,
#                                                   stratify=df.category.values)

In [40]:
# df.index.values

In [41]:
# df['data_type'] = ['not_set']*df.shape[0]

In [42]:
# df.head()

In [43]:
# df.loc[X_train, 'data_type'] = 'train'
# df.loc[X_val, 'data_type'] = 'val'

In [48]:
# df.groupby(['category', 'data_type']).count()
df.head()

,text,category,stance
0,بيل غيتس يتلقى لقاح #كوفيد19 من غير تصوير الاب...,0,1
1,وزير الصحة لحد اليوم وتحديدا هلأ بمؤتمروا الصح...,1,1
2,قولكن رح يكونو اد المسؤولية ب لبنان لما يوصل ...,1,1
3,#تركيا.. وزير الصحة فخر الدين قوجة يتلقى أول ج...,0,1
4,وئام وهاب يشتم الدول الخليجية في كل طلة اعلامي...,2,0


Loading Tokenizer and Encoding our Data

In [49]:
from transformers import BertTokenizer
from torch.utils.data import TensorDataset

In [50]:
tokenizer = BertTokenizer.from_pretrained(
    'bert-base-uncased',
    do_lower_case=True
)

In [51]:
#df[df.data_type=='train'].text.values
encoded_data_train = tokenizer.batch_encode_plus(
    df['text'].values,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=256,
    return_tensors='pt'
)


array(['بيل غيتس يتلقى لقاح #كوفيد19 من غير تصوير الابرة و لا السيرنجة و لا الدواء و لابس بولو صيفي في عز الشتاء و يقول ان إحدى مزايا عمر ال 65 عامًا هي انه مؤهل للحصول على اللقاح ... يعنى ما كان يحتاج اللقاح لو كان عمره اصغر من 65 🤔 https://t.co/QQKFFUNwBn',
       'وزير الصحة لحد اليوم وتحديدا هلأ بمؤتمروا الصحفي كان ما عندي مشكلة معوا.<LF>بس انوا بفايزر هو اللقاح الوحيد الآمن بالعالم لأ حبيبي هاي فيها نفاق واضح.<LF>عفوا يعني.<LF>مش إنوا مجبورين فيه وما قادر تجبر الدولة المارقة تدفع مصاري تجيب لقاح تاني.<LF>يلا باي',
       'قولكن  رح يكونو اد المسؤولية ب لبنان لما يوصل اللقاح؟<LF>ألفي جرعة من لقاح #كورونا للتلف في مستشفى بوسطن،لأن عامل النظافة وقف عن طريق الخطأ عمل الثلاجة التي حفظت بها.<LF>#لقاح_كورونا',
       ..., 'الجرعة الاولى من لقاح كورونا 👌 https://t.co/L2a4uR1Zsb',
       'بعد حظر #خامنئي اللقاحات الغربية.. #طهران بصدد إجازة تجربة لقاح كوبي على الإيرانيين https://t.co/3WbWhZQFJF',
       'مع تصاعد القلق إثر حالات وفاة بين متلقي لقاح بيونتيك وفايزر، أكدت وكالة الأدوية التابع